## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-27-22-53-32 +0000,nypost,Toronto mayor mercilessly mocked for on-air sn...,https://nypost.com/2026/01/27/world-news/toron...
1,2026-01-27-22-51-00 +0000,wsj,"As a partial government shutdown looms, Democr...",https://www.wsj.com/politics/policy/senators-s...
2,2026-01-27-22-49-00 +0000,wsj,Opinion | A Big Social-Media Shakedown,https://www.wsj.com/opinion/social-media-lawsu...
3,2026-01-27-22-49-00 +0000,wsj,Dollar Extends Slide After Trump Says He Isn’t...,https://www.wsj.com/finance/currencies/dollar-...
4,2026-01-27-22-46-00 +0000,wsj,Opinion | America’s Immigration Labor Shock,https://www.wsj.com/opinion/u-s-immigration-po...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2464/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,68
12,ice,24
58,minnesota,21
482,minneapolis,18
56,year,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
27,2026-01-27-22-08-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...,161
283,2026-01-27-10-28-15 +0000,cbc,Judge in Minnesota orders acting ICE director ...,https://www.cbc.ca/news/world/bovino-ice-minne...,146
332,2026-01-27-01-29-20 +0000,nypost,ICE tensions could be thawing in Minn. after T...,https://nypost.com/2026/01/26/us-news/ice-tens...,139
258,2026-01-27-12-02-00 +0000,wsj,Less than 48 hours after U.S. agents fatally s...,https://www.wsj.com/politics/policy/homan-in-b...,139
323,2026-01-27-03-01-00 +0000,wsj,"Democratic Rep. Ilhan Omar of Minnesota, facin...",https://www.wsj.com/politics/policy/minnesota-...,138


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
27,161,2026-01-27-22-08-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...
96,78,2026-01-27-20-11-28 +0000,nyt,Photo Shows ICE Agents Deploying at Minneapoli...,https://www.nytimes.com/2026/01/27/us/minneapo...
198,67,2026-01-27-15-59-40 +0000,nypost,Meta and YouTube face landmark trial over yout...,https://nypost.com/2026/01/27/business/meta-ti...
343,62,2026-01-27-00-42-00 +0000,wsj,The Army will gain access to Salesforce’s tech...,https://www.wsj.com/politics/national-security...
242,54,2026-01-27-13-22-00 +0000,wsj,EU and India Reach Free-Trade Deal as World Re...,https://www.wsj.com/economy/trade/eu-and-india...
45,47,2026-01-27-21-45-30 +0000,latimes,California population growth flattens amid Tru...,https://www.latimes.com/california/story/2026-...
10,46,2026-01-27-22-37-07 +0000,nypost,Alleged psycho surgeon Michael McKee lurked ou...,https://nypost.com/2026/01/27/us-news/alleged-...
110,43,2026-01-27-19-57-28 +0000,nypost,"Bari Weiss lays out vision for CBS News, shoot...",https://nypost.com/2026/01/27/media/bari-weiss...
319,42,2026-01-27-03-36-00 +0000,wsj,"Deadly Winter Storm Disrupts Air Travel, Leave...",https://www.wsj.com/us-news/climate-environmen...
332,39,2026-01-27-01-29-20 +0000,nypost,ICE tensions could be thawing in Minn. after T...,https://nypost.com/2026/01/26/us-news/ice-tens...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
